# Explaining Distribution Shifts in Time using Random Forests and t-SNE

In this Python notebook, we use machine learning techniques to explain a distribution shift in a time series. Specifically, we aim to identify which data points are specific to the time before or after the distribution shift, as well as which data points are distributed independently of time.

To achieve this goal, we train a random forest (RF) model on the MNIST dataset, which contains handwritten digits. MNIST is a classic dataset in machine learning that is often used as a benchmark for image classification tasks, as it presents a challenging high-dimensional classification problem with various classes to choose from. The random forest model is a versatile and robust model that can handle a wide range of input data and is relatively insensitive to weak data assumptions. The model is well-suited to handle the complex and high-dimensional input data in the MNIST dataset, making it a good choice for this machine learning task.

To visualize the results, we use t-SNE to project the high-dimensional data onto a two-dimensional plot. The t-SNE algorithm is used to derive an embedding space from the information that is extracted during the random forest training. The plot shows how clusters change before and after the distribution shift, providing insights to better understand the nature of the shift. By visualizing the embedding space, we can see which data points are specific to the time before or after the distribution shift and how the clusters change over time. This information can be used to improve the performance of the random forest model and gain a better understanding of the data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy.sparse.linalg import eigs
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

## Exploring the MNIST Dataset: Mean Images, Similarity Matrix, and t-SNE Visualization

In this part, we explore the MNIST dataset using mean images, similarity, and t-SNE. We first compute the mean image for each digit class and use it to compute the similarity between each pair of digit classes. We then visualize the similarity matrix using matplotlib. Finally, we use t-SNE to project the high-dimensional data onto a two-dimensional space, which allows us to visualize the dataset and gain insights into its structure.

In [ ]:
print('Downloading the dataset...')
mnist = fetch_openml('mnist_784')
print('Download complete.')

# Store the dataset in an appropriate format
X = np.array(mnist.data).reshape((-1, 28, 28))
y = np.array(mnist.target).astype(int)

In [ ]:
# Create an empty numpy array to store the mean images for each label
Xmean = np.zeros((len(set(y)), 28, 28))

# Iterate over each label in the dataset
for i in set(y):
    # Compute the mean image for the current label
    Xmean[i] = X[y == i].mean(axis=0)
    
# Create a figure with two rows and five columns to plot class means
fig, axs = plt.subplots(nrows=2, ncols=5, figsize=(10, 3))

# Iterate over each subplot and plot the data
for i in range(2):
    for j in range(5):
        label = i * 5 + j
        axs[i, j].axis('off')
        axs[i, j].matshow(Xmean[label])
        axs[i, j].set_title('label {} mean:'.format(label))

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.3, hspace=0.7)
plt.show()

In [ ]:
# Create an empty numpy array to store the dot product between the mean images
scalar = np.zeros((Xmean.shape[0], Xmean.shape[0]))

# Iterate over each pair of labels and compute the dot product between their mean images
for i in range(Xmean.shape[0]):
    for j in range(Xmean.shape[0]):
        scalar[i, j] = Xmean[i].flatten() @ Xmean[j].flatten()

# Normalize the similarity matrix so that its rows and columns sum to 1
scalar = scalar / (scalar.sum(axis=0)[:, None] * scalar.sum(axis=1)[None, :])

# Display the similarity matrix using matplotlib
plt.matshow(scalar)
plt.xlabel('Digit Class')
plt.xticks(range(10))
plt.ylabel('Digit Class')
plt.yticks(range(10))
plt.title('Similarity matrix between mean images')

plt.show()

In [ ]:
# Randomly select 5000 samples from the dataset
embedding_selection = np.random.choice(range(X.shape[0]), size=5000, replace=False)

# Fit the t-SNE model to the selected samples and obtain the low-dimensional embeddings
print('Fitting the t-SNE model...')
tsne = TSNE(init='random', learning_rate='auto')
X_embedded = tsne.fit_transform(X[embedding_selection].reshape(-1, 28*28))
print('Fitting complete.')

In [ ]:
# Set the figure size to 10x10 inches
plt.figure(figsize=(10, 10))

# Specify the percentage of samples to display for each label
percentage = 30

# Iterate over each label in the dataset
for i in set(y):
    # Select the samples with the current label
    selection_label = y[embedding_selection] == i
    # Calculate the number of samples to display based on the specified percentage
    num_samples = int(np.ceil(np.sum(selection_label) * percentage / 100))
    # Select a random subset of the samples with the current label
    selection_random = np.random.choice(np.where(selection_label)[0], size=num_samples, replace=False)
    # Create a scatter plot of the selected samples, with the label as the marker and alpha set to 0.5
    plt.scatter(X_embedded[selection_random, 0], X_embedded[selection_random, 1], marker="$%i$" % i, alpha=0.5, s=25)

# Display the plot
plt.title(f't-SNE visualization of MNIST ({percentage}% of the dataset)')
plt.show()

## Preparing the data for the distribution shift analysis

In this part, we prepare the MNIST dataset for the distribution shift analysis. We map each digit in the dataset to a label indicating whether it occurs before or after the change point, or both, or neither. We then remove the samples with label 0 (i.e., digits that do not occur before or after the change point) from the input features and labels. Finally, we randomly assign labels of 1 or 2 to samples with label 3 (i.e., digits that occur both before and after the change point). This ensures that we have a balanced dataset with labels 1 and 2 representing the digits that occur before and after the change point, respectively.

In [ ]:
# Map each digit to a label indicating whether it occurs before or after the change point, or both, or neither
#  0 - never, 1 - before, 2 - after, 3 - both
label_map = {
    0: 0,
    1: 1,
    2: 0,
    3: 1,
    4: 3,
    5: 0,
    6: 0,
    7: 2,
    8: 2,
    9: 0
}
y_mapped = np.array([label_map[digit] for digit in y])

# Remove samples with label 0 (i.e., digits that do not occur before or after the change point) 
#  from input features and labels
X_clean = X.copy().reshape(-1, 28*28)[y_mapped != 0]
y_clean = y.copy()[y_mapped != 0]
y_mapped_clean = y_mapped.copy()[y_mapped != 0]

# Randomly assign labels of 1 or 2 to samples with label 3
#  (i.e., digits that occur both before and after the change point)
label_3_idx = np.where(y_mapped_clean == 3)[0]
y_mixed = y_mapped_clean.copy()
y_mixed[label_3_idx] = np.random.choice([1, 2], size=len(label_3_idx))

## Computing the RF-Kernel Matrix and Visualizing Learned Similarities Between Digits

In this part, we train a random forest classifier on the mixed training set of digits, where the digits that occur both before and after the distribution shift have been randomly assigned to either the before or after group. We use the trained classifier to compute an RF-kernel-matrix for a subset of the input features and labels. We then display the RF-kernel-matrix, where the samples are grouped by digit class.

In [ ]:
# Set a boolean flag to determine whether to skip model selection and train with max_leaf_nodes=150
skip_model_selection = True

# Split the dataset into a training set and a test set, with a 55/45 split
X_clean_train, X_clean_test, y_mixed_train, y_mixed_test = \
    train_test_split(X_clean.reshape(-1,28*28), y_mixed, train_size=0.55)

# Initialize a random forest model with max_leaf_nodes=150
model = RandomForestClassifier(max_leaf_nodes=150)

# Train the model on the mixed training set (group 3 randomly assigned to 1 or 2)
if not skip_model_selection:
    best_model, best_score = None, -5
    start_nodes, end_nodes, step_size = 50, 200, 5
    for max_leaf_nodes in range(start_nodes, end_nodes+1, step_size):
        # Train a random forest model with the current value of max_leaf_nodes
        rf_model = RandomForestClassifier(max_leaf_nodes=max_leaf_nodes).fit(X_clean_train, y_mixed_train)
        # Evaluate the model on the test set and store the score
        test_score = rf_model.score(X_clean_test, y_mixed_test)
        print(f"Test set accuracy for max_leaf_nodes = {max_leaf_nodes}: {test_score:.3f}")
        # If the current model has a higher score than the previous best model, update the best model and best score
        if test_score > best_score:
            best_model = rf_model
            best_score = test_score
    print("----")
    print(f"Best test set accuracy: {best_score:.3f}")
    # Set the model to the best model found during model selection
    model = best_model

# Fit the model to the mixed set (group 3 is randomly assigned to 1 or 2)
print('Fitting Random Forest classifier...')
model.fit(X_clean, y_mixed);
print('Fitting complete.')

In [ ]:
# Select a random subset of the clean input features and labels
subset_size = 5000
subset_indices = np.random.choice(range(X_clean.shape[0]), size=subset_size, replace=False)
X_subset, y_subset = X_clean[subset_indices], y_clean[subset_indices]

# Compute the RF-Kernel-Matrix for the subset using the trained random forest model
print('Computing Random Forest kernel matrix...')
rf_kernel_matrix = np.zeros((subset_size, subset_size))

leaf_indices = model.apply(X_subset.reshape(-1,28*28))
for leaf_vector in leaf_indices.T:
    # Compute the pairwise similarity between leaf indices using boolean array comparison
    rf_kernel_matrix += leaf_vector[:, None] == leaf_vector[None, :]

# Normalize the RF-Kernel-Matrix by the number of decision trees in the random forest
rf_kernel_matrix /= leaf_indices.shape[1]
print('Computing complete.')

In [ ]:
# Sort the subset of input features and labels by their ground truth class
sorted_indices_by_class = np.argsort(y_subset)

# Display the RF-kernel matrix, with samples grouped by class
plt.set_cmap("viridis")
plt.matshow(rf_kernel_matrix[sorted_indices_by_class, :][:, sorted_indices_by_class])

# Get the unique labels and their counts in the sorted subset
unique_labels, label_counts = np.unique(y_subset, return_counts=True)

# Sort the unique labels and their counts by label
sort_indices = np.argsort(unique_labels)
unique_labels = unique_labels[sort_indices]
label_counts = label_counts[sort_indices]

# Calculate the midpoint of each group of samples
midpoints = np.cumsum(label_counts) - label_counts / 2 - 1

# Add x and y ticks with labels for each groupplt.xticks(midpoints, unique_labels)
plt.xticks(midpoints, unique_labels)
plt.xlabel('Digit Class')
plt.yticks(midpoints, unique_labels)
plt.ylabel('Digit Class')

# Add a title
plt.title('Similarities from RF classifier sorted by digits')
plt.show();

This plot visualizes the computed similarity between pairs of samples derived from the classification model. High similarity is encoded by bright colors. The plot shows the samples sorted by their ground truth digit classes 1, 3, 4, 7, and 8, which were mapped to labels indicating whether they occurred before or after the change point, or both. The original digit classes are unknown to the classifier. We can see that the model is able to distinguish well between the two main groups, formed by digits 1 and 3 (group 1) and digits 7 and 8 (group 2). Note that even though digit 4 was randomly assigned to either group 1 or 2, the model was able to separate this class from the others very clearly.

## t-SNE Visualization of Random Forest Kernel Matrix Embeddings with Grouped Digits

In the previous section, we computed the similarity matrix of the random forest classifier and plotted it. In this section, we apply t-SNE to the first five principal components of the eigenvectors of the RF-kernel-matrix and visualize the results. We show that the resulting plot exhibits clear cluster structure, corresponding to the different digit types. We also show that the color of each point in the plot can be interpreted as the probability of the point occurring before or after the change point. This plot allows us to visually explore the relationship between the digit types and their temporal context.

In [ ]:
# Compute the eigenvalues and eigenvectors of the RF-kernel-matrix
print('Computing eigenvalues and eigenvectors...')
embedding_eigenvalues, embedding_eigenvectors = eigs(rf_kernel_matrix, k=5, which='LM')
print('Computing eigenvalues and eigenvectors complete.')

# Apply t-SNE to the first 5 principal components of the eigenvectors of the RF-Kernel-Matrix
print('Fitting t-SNE model...')
tsne_model = TSNE(init='random', learning_rate='auto')
tsne_embedding = tsne_model.fit_transform(np.real((embedding_eigenvectors * (embedding_eigenvalues**0.5)[None,:])))
print('Fitting complete.')

In [ ]:
# Specify the percentage of samples to display for each label
percentage = 30

# Set the colormap to 'coolwarm' and plot the samples using different markers and colors for each label
plt.figure(figsize=(12,10))
plt.set_cmap("coolwarm")
for label in set(y_subset):
    selection_label = y_subset == label
    # Calculate the number of samples to display based on the specified percentage
    num_samples = int(np.ceil(np.sum(selection_label) * percentage / 100))
    # Select a random subset of the samples with the current label
    selection_random = np.random.choice(np.where(selection_label)[0], size=num_samples, replace=False)
    if selection_label.sum() > 0:
        # Plot the samples with the current label, using the label as the marker and the model's predicted probability 
        #  as the color
        plt.scatter(tsne_embedding[selection_random][:,0], tsne_embedding[selection_random][:,1], marker="$%i$" % label, 
                    c=model.predict_proba(X_subset[selection_random].reshape(-1,28*28))[:,0], alpha=0.5, s=40)

# Display the plot
plt.xlabel("t-SNE dimension 1")
plt.ylabel("t-SNE dimension 2")
plt.title(f't-SNE Embedding of MNIST Data Colored by Random Forest Predictions ({percentage}% of the dataset)')
plt.colorbar()
plt.show()

The t-SNE plot shows a clear clustering structure according to the digit types. Each cluster can be identified in its time context by the color that is computed from the RF prediction. This color coding gives rise to the two time dependent groups formed by digits 1 and 3 (group 1) and digits 7 and 8 (group 2). Each of the five digits, in particular the randomly assigned digit 4, is well distinguishable in the clustering structure. Note that the structure that we obtained by the kernel analysis has two main advantages over solely evaluating the color coded class probability: It helps us distinguish between different kinds of data points that would be indistinguishable when looking at the predicted class probabilities only. Furthermore it helps identify time-independent clusters even if the classifier does not predict each of its samples exactly with 0.5.